# Abstractive Text Summarization Using T5
_By Ling Li Ya_

References:
1. [Exploring the Limits of Transfer Learning with a Unified
Text-to-Text Transformer](https://arxiv.org/abs/1910.10683)
2. [Truecasing in Natural Language Processing](https://towardsdatascience.com/truecasing-in-natural-language-processing-12c4df086c21)
3. [POS Tag List Reference](https://stackoverflow.com/questions/29332851/what-does-nn-vbd-in-dt-nns-rb-means-in-nltk)

## 1. Install and Import Dependencies
Install `pytorch`

In [ ]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


You should consider upgrading via the 'C:\Users\liana\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Install `transformers` to use its `summarization pipeline`

In [3]:
!pip install transformers

You should consider upgrading via the 'C:\Users\liana\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Install `bs4` to use `BeautifulSoup`

In [4]:
!pip install bs4

You should consider upgrading via the 'C:\Users\liana\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Install `standfordnlp` to use its `POS processor pipeline`

In [5]:
!pip install stanfordnlp

You should consider upgrading via the 'C:\Users\liana\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Import all dependencies

In [6]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests
import stanfordnlp

## 2. Setup Generator
Get pipeline text summarization utility

In [7]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="pt")

## 3. Process Input Text
Get input from website URLs

In [8]:
URL = 'https://en.wikipedia.org/wiki/Rococo'

Get HTTP URL using `requests`

In [9]:
r = requests.get(URL)

Parse HTML body returned from the URL and format it to have a better readability

In [10]:
soup = BeautifulSoup(r.text, 'html.parser')

Find all text chunks with 'h1' and 'p' tags

In [11]:
results = soup.find_all(['h1', 'p'])
results[:3]

[<h1 class="firstHeading" id="firstHeading">Rococo</h1>,
 <p class="mw-empty-elt">
 </p>,
 <p><b>Rococo</b> (<span class="rt-commentedText nowrap"><span class="IPA nopopups noexcerpt"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="'r' in 'rye'">r</span><span title="/ə/: 'a' in 'about'">ə</span><span title="/ˈ/: primary stress follows">ˈ</span><span title="'k' in 'kind'">k</span><span title="/oʊ/: 'o' in 'code'">oʊ</span><span title="'k' in 'kind'">k</span><span title="/oʊ/: 'o' in 'code'">oʊ</span></span>/</a></span></span>, <small>also</small> <span class="rt-commentedText nowrap"><small><a href="/wiki/American_English" title="American English">US</a>: </small><span class="IPA nopopups noexcerpt"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="/ˌ/: secondary stress follows">ˌ</span><span title="'r' in 'rye'">r</span><span title="/oʊ/: 'o' in 'code'">oʊ</span><

Text enclosed within the HTML tags are selected and joined together

In [12]:
text = [result.text for result in results]
ARTICLE = ' '.join(text)
ARTICLE[0:1000]

"Rococo \n Rococo (/rəˈkoʊkoʊ/, also US: /ˌroʊkəˈkoʊ/), less commonly Roccoco or Late Baroque, is an exceptionally ornamental and theatrical style of architecture, art and decoration which combines asymmetry, scrolling curves, gilding, white and pastel colors, sculpted molding, and trompe-l'œil frescoes to create surprise and the illusion of motion and drama.  It is often described as the final expression of the Baroque movement.[1]\n The Rococo style began in France in the 1730s as a reaction against the more formal and geometric Style Louis XIV. It was known as the style rocaille, or rocaille style.[2] It soon spread to other parts of Europe, particularly northern Italy, Austria, southern Germany, Central Europe and Russia.[3] It also came to influence the other arts, particularly sculpture, furniture, silverware, glassware, painting, music, and theatre.[4] Although originally a secular style primarily used for interiors of private residences the Rococo had a spiritual aspect to it w

## 4. Chunk text

Append <eos> to punctuations that marks the end of a sentence
<br />
Without the <eos> tag, sentences will be split without any punctuation

In [13]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
sentences = ARTICLE.split('<eos>')
sentences[:10]

["Rococo \n Rococo (/rəˈkoʊkoʊ/, also US: /ˌroʊkəˈkoʊ/), less commonly Roccoco or Late Baroque, is an exceptionally ornamental and theatrical style of architecture, art and decoration which combines asymmetry, scrolling curves, gilding, white and pastel colors, sculpted molding, and trompe-l'œil frescoes to create surprise and the illusion of motion and drama.",
 '  It is often described as the final expression of the Baroque movement.',
 '[1]\n The Rococo style began in France in the 1730s as a reaction against the more formal and geometric Style Louis XIV.',
 ' It was known as the style rocaille, or rocaille style.',
 '[2] It soon spread to other parts of Europe, particularly northern Italy, Austria, southern Germany, Central Europe and Russia.',
 '[3] It also came to influence the other arts, particularly sculpture, furniture, silverware, glassware, painting, music, and theatre.',
 '[4] Although originally a secular style primarily used for interiors of private residences the Rococo

Limit the size of text in a chunk so that it is smaller than 500 words
<br />
Split sentences into words (2D array)
<br />
This is to avoid the error as shown below
```py
Token indices sequence length is longer than the specified maximum sequence length for this model (1024). Running this sequence through the model will result in indexing errors.
```

In [14]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        # Check if the chunk is less than 500 words
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        # Next chunk
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

print("A total of " + str(current_chunk + 1) + " chunks")
print("A total of " + str(len(chunks[0])) + " words in chunk[0]")

0
A total of 12 chunks
A total of 488 words in chunk[0]


Append words into sentences again where each chunk is ensured to have less than 500 words

In [15]:
for chunk_id in range (len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

print("A total of " + str(len(chunks[0].split(' '))) + " words in chunk[0]")

A total of 488 words in chunk[0]


## 5. Summarise Text

Summarise based on each chunk

In [16]:
res = summarizer(chunks)
res

Token indices sequence length is longer than the specified maximum sequence length for this model (893 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'the Rococo style began in the 1730s as a reaction against the more formal and geometric Style Louis XIV . the term rocaille was first used in print in 1825 to describe decoration which was "out of style and old-fashioned" since the mid-19th century, the term has been accepted by art historians .'},
 {'summary_text': 'the Rocaille style, or French Rococo, appeared in Paris during the reign of Louis XV . it flourished between about 1723 and 1759, and was used particularly in salons . the furniture featured sinuous curves and vegetal designs .'},
 {'summary_text': 'the Venetian Rococo style is deeply anchored in popular culture . it is characterized by a light-filled weightlessness, festive cheerfulness and movement . the style reached its peak in southern Germany and Austria from the 1730s until the 1770s .'},
 {'summary_text': 'François de Cuvilliés was one of the first to create a Rococo building in Germany . the pavilion of Amalienburg in Munich was inspired by the 

## 6. Formatting Text
Preprocessing: format the `dict` object into a `string`.

In [17]:
summary = ''
for result in res:
    summary += ''.join(str(val.capitalize()) + "\n" for _, val in result.items())

summary = summary.replace(' .', '.')
summary = summary.replace(" !", "!")
summary = summary.replace(" ?", "?")

Check `pytorch` version. `standfordnlp` requires at least version 1.0.0 or older.

`torch==version_number`

In [18]:
!pip freeze | grep torch

pytorch-lightning==1.4.5
torch==1.9.0+cu111
torchaudio==0.9.0
torchmetrics==0.5.1
torchvision==0.10.0+cu111


Download the English `stanfordnlp` model. It will take some time because the model is very huge (about 1.96GB). Type 'y' to continue with the download in any prompted dialogue box.

In [19]:
# stanfordnlp.download('en')

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)

Default download directory: C:\Users\liana\stanfordnlp_resources
Hit enter to continue or type an alternate directory.

Download location: C:\Users\liana\stanfordnlp_resources\en_ewt_models.zip


100%|██████████| 235M/235M [02:02<00:00, 1.92MB/s]



Download complete.  Models saved to: C:\Users\liana\stanfordnlp_resources\en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


Create a `pipeline` with `pos` processor. POS stands for Parts of speech where tagging is done.

In [20]:
stf_nlp = stanfordnlp.Pipeline(processors='tokenize,mwt,pos')

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\liana\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\liana\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': 'C:\\Users\\liana\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


Evaluate the preprocessed summary text with `stanfordnlp`.

In [21]:
doc = stf_nlp(summary)

The breakdown analysis of the summary text.

In [22]:
print(*[f'word: {word.text+" "}\tupos: {word.upos}\txpos: {word.xpos}' for sent in doc.sentences for word in sent.words], sep='\n')

word: The 	upos: DET	xpos: DT
word: rococo 	upos: ADJ	xpos: JJ
word: style 	upos: NOUN	xpos: NN
word: began 	upos: VERB	xpos: VBD
word: in 	upos: ADP	xpos: IN
word: the 	upos: DET	xpos: DT
word: 1730s 	upos: NOUN	xpos: NNS
word: as 	upos: ADP	xpos: IN
word: a 	upos: DET	xpos: DT
word: reaction 	upos: NOUN	xpos: NN
word: against 	upos: ADP	xpos: IN
word: the 	upos: DET	xpos: DT
word: more 	upos: ADV	xpos: RBR
word: formal 	upos: ADJ	xpos: JJ
word: and 	upos: CCONJ	xpos: CC
word: geometric 	upos: ADJ	xpos: JJ
word: style 	upos: NOUN	xpos: NN
word: louis 	upos: PROPN	xpos: NNP
word: xiv. 	upos: NOUN	xpos: NN
word: the 	upos: DET	xpos: DT
word: term 	upos: NOUN	xpos: NN
word: rocaille 	upos: NOUN	xpos: NN
word: was 	upos: AUX	xpos: VBD
word: first 	upos: ADV	xpos: RB
word: used 	upos: VERB	xpos: VBN
word: in 	upos: ADP	xpos: IN
word: print 	upos: NOUN	xpos: NN
word: in 	upos: ADP	xpos: IN
word: 1825 	upos: NUM	xpos: CD
word: to 	upos: PART	xpos: TO
word: describe 	upos: VERB	xpos: VB
word:

Capitalise proper nouns. `PROPN` stands for proper noun and `NNP` stands for proper noun, singular phrase.

In [23]:
doc_list = [w.text.capitalize() if w.upos in ["PROPN","NNS"] else w.text for sent in doc.sentences for w in sent.words]
doc_list

['The',
 'rococo',
 'style',
 'began',
 'in',
 'the',
 '1730s',
 'as',
 'a',
 'reaction',
 'against',
 'the',
 'more',
 'formal',
 'and',
 'geometric',
 'style',
 'Louis',
 'xiv.',
 'the',
 'term',
 'rocaille',
 'was',
 'first',
 'used',
 'in',
 'print',
 'in',
 '1825',
 'to',
 'describe',
 'decoration',
 'which',
 'was',
 '"',
 'out',
 'of',
 'style',
 'and',
 'old',
 '-',
 'fashioned',
 '"',
 'since',
 'the',
 'mid-19th',
 'century',
 ',',
 'the',
 'term',
 'has',
 'been',
 'accepted',
 'by',
 'art',
 'historians',
 '.',
 'The',
 'rocaille',
 'style',
 ',',
 'or',
 'french',
 'rococo',
 ',',
 'appeared',
 'in',
 'Paris',
 'during',
 'the',
 'reign',
 'of',
 'Louis',
 'Xv.',
 'it',
 'flourished',
 'between',
 'about',
 '1723',
 'and',
 '1759',
 ',',
 'and',
 'was',
 'used',
 'particularly',
 'in',
 'salons',
 '.',
 'the',
 'furniture',
 'featured',
 'sinuous',
 'curves',
 'and',
 'vegetal',
 'designs',
 '.',
 'The',
 'venetian',
 'rococo',
 'style',
 'is',
 'deeply',
 'anchored',
 'in

Capitalise every first word of the sentence. Add a space in front of words that are not punctuation.

In [24]:
i = 0
for sent in doc.sentences:
    for w in range(len(sent.words)):
        if w != 2:
            if sent.words[w - 1].xpos in ["!", "."]: # Capitalise each first word
                doc_list[i] = sent.words[w].text.capitalize()
        if sent.words[w].upos != "PUNCT" and i != 0: # Add a space before non-punctuation words
            doc_list[i] = " "+ doc_list[i]
        i += 1

doc_list

['The',
 ' rococo',
 ' style',
 ' began',
 ' in',
 ' the',
 ' 1730s',
 ' as',
 ' a',
 ' reaction',
 ' against',
 ' the',
 ' more',
 ' formal',
 ' and',
 ' geometric',
 ' style',
 ' Louis',
 ' xiv.',
 ' the',
 ' term',
 ' rocaille',
 ' was',
 ' first',
 ' used',
 ' in',
 ' print',
 ' in',
 ' 1825',
 ' to',
 ' describe',
 ' decoration',
 ' which',
 ' was',
 '"',
 ' out',
 ' of',
 ' style',
 ' and',
 ' old',
 '-',
 ' fashioned',
 '"',
 ' since',
 ' the',
 ' mid-19th',
 ' century',
 ',',
 ' the',
 ' term',
 ' has',
 ' been',
 ' accepted',
 ' by',
 ' art',
 ' historians',
 '.',
 ' The',
 ' rocaille',
 ' style',
 ',',
 ' or',
 ' french',
 ' rococo',
 ',',
 ' appeared',
 ' in',
 ' Paris',
 ' during',
 ' the',
 ' reign',
 ' of',
 ' Louis',
 ' Xv.',
 ' it',
 ' flourished',
 ' between',
 ' about',
 ' 1723',
 ' and',
 ' 1759',
 ',',
 ' and',
 ' was',
 ' used',
 ' particularly',
 ' in',
 ' salons',
 '.',
 ' The',
 ' furniture',
 ' featured',
 ' sinuous',
 ' curves',
 ' and',
 ' vegetal',
 ' design

Join all items in `doc_list` into a string.

In [25]:
summary = ""
for s in doc_list:
    summary += s

## 7. Results

Some statistics and the final result.

In [48]:
sentences = []
one_sen = ""
for s in doc_list:
    one_sen += s
    if s == ".":
        sentences.append(one_sen)
        one_sen = ""

sentences

['The rococo style began in the 1730s as a reaction against the more formal and geometric style Louis xiv. the term rocaille was first used in print in 1825 to describe decoration which was" out of style and old- fashioned" since the mid-19th century, the term has been accepted by art historians.',
 ' The rocaille style, or french rococo, appeared in Paris during the reign of Louis Xv. it flourished between about 1723 and 1759, and was used particularly in salons.',
 ' The furniture featured sinuous curves and vegetal designs.',
 ' The venetian rococo style is deeply anchored in popular culture.',
 ' It is characterized by a light- filled weightlessness, festive cheerfulness and movement.',
 ' The style reached its peak in southern Germany and Austria from the 1730s until the 1770s.',
 ' François De Cuvilliés was one of the first to create a rococo building in Germany.',
 ' The pavilion of amalienburg in Munich was inspired by the pavilions of the trianon and marly.',
 ' A number of no

In [50]:
md = "" # Markdown output to be fed into slide generator
txt = ""
for sentence in sentences:
        if(len(txt) == 0):
            md += "# " + "Header" + "\n\n- " + sentence
            txt += sentence
        elif(len(txt) < 1000 and sentence != sentences[-1]): # If the number of characters on a slide exceeds 1000, create new slide
            md += "\n-" + sentence
            txt += sentence
        else:
            md += "\n\n---\n\n"
            txt = ""

print(md)

# Header

- The rococo style began in the 1730s as a reaction against the more formal and geometric style Louis xiv. the term rocaille was first used in print in 1825 to describe decoration which was" out of style and old- fashioned" since the mid-19th century, the term has been accepted by art historians.
- The rocaille style, or french rococo, appeared in Paris during the reign of Louis Xv. it flourished between about 1723 and 1759, and was used particularly in salons.
- The furniture featured sinuous curves and vegetal designs.
- The venetian rococo style is deeply anchored in popular culture.
- It is characterized by a light- filled weightlessness, festive cheerfulness and movement.
- The style reached its peak in southern Germany and Austria from the 1730s until the 1770s.
- François De Cuvilliés was one of the first to create a rococo building in Germany.
- The pavilion of amalienburg in Munich was inspired by the pavilions of the trianon and marly.
- A number of notable pilgrima

In [ ]:
words_after = len(summary.split(' '))
words_before = len(ARTICLE)
reduced_by = (words_before - words_after) / words_before * 100

print("Number of words in summary: " + str(words_after))
print("Number of words in original article: " + str(words_before))
print("Reduced by: " + str(round(reduced_by, 2)) + "%\n")
print(summary)

Number of words in summary: 500
Number of words in original article: 37336
Reduced by: 98.66%

The rococo style began in the 1730s as a reaction against the more formal and geometric style Louis xiv. the term rocaille was first used in print in 1825 to describe decoration which was" out of style and old- fashioned" since the mid-19th century, the term has been accepted by art historians. The rocaille style, or french rococo, appeared in Paris during the reign of Louis Xv. It was used particularly in salons, a new style of room designed to impress and entertain guests. The furniture designers and craftsmen in the style included Juste- aurele meissonier and Nicolas Pineau. The venetian rococo style is deeply anchored in popular culture. It is characterized by a light- filled weightlessness, festive cheerfulness and movement. The style reached its peak in southern Germany and Austria from the 1730s until the 1770s. François De Cuvilliés was one of the first to create a rococo building in 